In [1]:
%load_ext autoreload
%autoreload 2

%pylab inline



Populating the interactive namespace from numpy and matplotlib


Using Theano backend.


In [2]:
import dataloaders
import models
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics 
import sklearn.cluster as cluster
import numpy as np 
import random
import classifiers
from torch.autograd import Variable
import torch
import torch.nn as nn 
import torch.optim as optim
import util
from pathlib import Path

from imblearn.under_sampling import RandomUnderSampler

In [72]:
number = 107
filename = f'house-{number}'
input_file_path = f'../data/house/{filename}-cleaned.csv'
input_file_path = '../data/slovenia/stranke94.csv'
delimiter = ','
data = dataloaders.UnsplitDataset(input_file_path, ratio=0.8, delimiter=delimiter)

In [73]:
X = data.X
y = data.y
n = data.get_num_nodes()

In [74]:
print('Number of nodes: ', n)
print('Number of +ve: ', len(y[y == 1]))
print('Number of -ve: ', len(y[y == 0]))
print('% of +ve: ', len(y[y == 1])/float(len(y)))
print('% of -ve: ', len(y[y == 0])/float(len(y)))

Number of nodes:  10
Number of +ve:  36
Number of -ve:  54
% of +ve:  0.4
% of -ve:  0.6


In [70]:
s = "1 0 0 0 0 -1 1 1 -1 -1 -1 1 -1 1 1 1 1 1 1 1 1 -1 1 1 1 1 -1 -1 1 1 1 1 1 1 1 -1 0 0 0 0 0 0 -1 -1 1 1 1 1 1 -1 1 -1 1 -1 1 -1 1 1 -1 1 1 1 1 1 -1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 -1 1 1 -1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 -1 1 -1 1 -1 1 1 1 1 1 0 0 0 1 -1 -1 1 1 1 -1 1 1 -1 1 1 1 -1 0 -1 1 -1 1 1 1 1 1 1 -1 1 1 -1 1 1 -1 1 -1 -1 -1 -1 1 1 1 -1 -1 -1 1 1 -1 1 1 1 -1 -1 1 -1 -1 -1 1 1 1 1 1 1 -1 1 1 1 1 -1 -1 1 1 -1 1 -1 -1 -1 1 1 1 1 1 1 -1 1 1 1 -1 -1 1 1 -1 1 1 -1 -1 -1 1 -1 1 1 1 -1 -1 -1 -1 1 -1 1 -1 -1 -1 1 -1 1 1 1 1 -1 1 1 1 1 -1 1 1 1 1 1 -1 -1 -1 -1 1 -1 -1 -1 1 1 1 1 1 -1 1 1 1 1 -1 1 -1 1 -1 1 1 1 1 1 1 1 1 1 -1 -1 -1 1 1 -1 -1 1 1 -1 1 -1 1 -1 1 1 1 1 1 -1 1 -1 -1 1 -1 -1 -1 1 1 1 1 1 -1 -1 1 1 1 1 1 0 -1 1 1 -1 1 1 -1 -1 1 1 1 1 1 1 1 -1 -1 1 1 -1 -1 -1 1 1 -1 1 -1 -1 1 1 1 -1 1 1 -1 -1 -1 -1 1 1 1 1 1 1 1 -1 -1 -1 1 1 1 -1 1 1 1 1 1 1 -1 -1 1 -1 -1 -1 1 -1 1 -1 -1 1 1 -1 1 -1 -1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 -1 1 -1 1 -1 -1 -1 1 1 1 1 -1 1 -1 1 1 -1 -1 1 1 1 -1 -1 1 1 1 1 -1 1 1 -1 1 1 1 1 -1 1 1 -1 -1 1 1 1 1 1 1 -1 -1 1 1 1 1 1 1 -1 -1 -1 -1 1 1 1 -1 -1 1 -1 -1 -1 -1 1 1 -1 -1 1 1 1 1 1 1 -1 1 1 1 -1 1 1 1 1 -1 1 -1 1 -1 -1 1 1 -1 -1 -1 1 -1 1 1 1 1 1 -1 1 1 -1 1 1 1 -1 1 1 -1 1 -1 1 1 -1 -1 1 1 -1 1 -1 1 1 1 -1 1 1 1 1 1 1 1 1 1 1 -1 1 -1 1 -1 -1 -1 -1 -1 -1 -1 1 1 1 -1 1 1 -1 -1 -1 -1 -1 -1 1 -1 1 -1 1 -1 1 -1 -1 1 -1 -1 1 -1 -1 -1 1 -1 1 -1 1 -1 1 -1 1 -1 1 -1 -1 -1 1 -1 -1 1 1 1 1 1 1 1 -1 1 -1 -1 1 -1 1 1 1 1 -1 1 -1 -1 1 1 1 -1 -1 -1 1 1 1 -1 -1 1 -1 1 -1 -1 1 1 -1 1 -1 1 -1 -1 1 -1 1 -1 1 -1 1 1 1 -1 -1 -1 -1 1 1 1 1 -1 -1 1 -1 -1 1 1 -1 1 -1 -1 -1 -1 -1 -1 -1 -1 -1 1 1 1 1 1 -1 -1 1 1 -1 0 0 -1 -1 -1 -1 -1 -1 -1 1 1 1 1 1 -1 1 -1 -1 -1 -1 1 1 -1 1 -1 -1 -1 1 1 -1 -1 -1 -1 -1 1 1 1 1 1 1 -1 -1 1 -1 1 1 -1 1 1 -1 -1 1 1 1 -1 1 1 1 1 -1 1 1 0 1 -1 1 1 1 -1 1 -1 -1 -1 1 -1 1 -1 1 1 -1 -1 1 1 1 1 1 1 1 -1 1 -1 0 -1 -1 1 1 1 1 1 1 1 -1 1 -1 1 1 1 1 -1 -1 1 -1 1 1 1 1 1 -1 1 1 1 1 1 1 1 1 -1 1 1 -1 1 1 -1 -1 1 1 1 -1 1 1 1 1 1 1 -1 -1 -1 -1 -1 -1 1 1 1 1 1 1 1 1 1 -1 1 1 1 -1 -1 1 -1 1 1 1 1 1 1 1 -1 1 1 1 1 1 -1 1 1 1 1 1 -1 1 1 1 1 1 1 1 1 -1 1 1 1 1 1 -1 -1 -1 1 -1 1 1 -1 -1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1"

In [71]:
len(s.split())

919

In [77]:
import numpy as np

In [75]:
d = {"102": {"stem": [{"micro_f1": 0.9886363636363636, "macro_f1": 0.9885640025990903}, {"micro_f1": 0.9772727272727273, "macro_f1": 0.9754464285714286}, {"micro_f1": 0.9886363636363636, "macro_f1": 0.9870263895031697}, {"micro_f1": 0.9545454545454546, "macro_f1": 0.9520697167755992}, {"micro_f1": 0.9772727272727273, "macro_f1": 0.9762803234501347}], "sine": [{"micro_f1": 0.9545454545454546, "macro_f1": 0.9539508110936683}, {"micro_f1": 0.9204545454545455, "macro_f1": 0.9074380165289255}, {"micro_f1": 0.8181818181818182, "macro_f1": 0.7498223169864962}, {"micro_f1": 0.8068181818181818, "macro_f1": 0.7794486215538847}, {"micro_f1": 0.8409090909090909, "macro_f1": 0.8199883109292812}]}, "103": {"stem": [{"micro_f1": 1.0, "macro_f1": 1.0}, {"micro_f1": 1.0, "macro_f1": 1.0}, {"micro_f1": 0.9772727272727273, "macro_f1": 0.9757575757575758}, {"micro_f1": 0.9659090909090909, "macro_f1": 0.9642421779764323}, {"micro_f1": 0.9772727272727273, "macro_f1": 0.9750990379173741}], "sine": [{"micro_f1": 0.9101123595505618, "macro_f1": 0.9098277608915908}, {"micro_f1": 0.9431818181818182, "macro_f1": 0.9429977976421815}, {"micro_f1": 0.9090909090909091, "macro_f1": 0.9051212938005391}, {"micro_f1": 0.8522727272727273, "macro_f1": 0.8478521079930843}, {"micro_f1": 0.9318181818181818, "macro_f1": 0.9228521332554063}]}, "104": {"stem": [{"micro_f1": 0.5280898876404494, "macro_f1": 0.34558823529411764}, {"micro_f1": 0.6292134831460674, "macro_f1": 0.38620689655172413}, {"micro_f1": 0.48314606741573035, "macro_f1": 0.32575757575757575}, {"micro_f1": 0.5168539325842697, "macro_f1": 0.3407407407407408}, {"micro_f1": 0.5, "macro_f1": 0.3333333333333333}], "sine": [{"micro_f1": 0.8764044943820225, "macro_f1": 0.8761543327008223}, {"micro_f1": 0.898876404494382, "macro_f1": 0.8894409937888199}, {"micro_f1": 0.9101123595505618, "macro_f1": 0.9101010101010101}, {"micro_f1": 0.9438202247191011, "macro_f1": 0.9435637285986049}, {"micro_f1": 0.7840909090909092, "macro_f1": 0.7827160493827161}]}, "105": {"stem": [{"micro_f1": 0.9775280898876404, "macro_f1": 0.9775252525252525}, {"micro_f1": 0.9887640449438202, "macro_f1": 0.9877156659765355}, {"micro_f1": 0.9887640449438202, "macro_f1": 0.9886725213185694}, {"micro_f1": 0.9659090909090909, "macro_f1": 0.9658694246929541}, {"micro_f1": 0.9772727272727273, "macro_f1": 0.9772609819121447}], "sine": [{"micro_f1": 0.8089887640449437, "macro_f1": 0.8086021505376343}, {"micro_f1": 0.9325842696629213, "macro_f1": 0.9277597402597402}, {"micro_f1": 0.898876404494382, "macro_f1": 0.8986717267552182}, {"micro_f1": 0.8636363636363636, "macro_f1": 0.8635658914728681}, {"micro_f1": 0.7386363636363636, "macro_f1": 0.7328052805280528}]}, "106": {"stem": [{"micro_f1": 0.9545454545454546, "macro_f1": 0.9545454545454545}, {"micro_f1": 0.9659090909090909, "macro_f1": 0.9629161399072903}, {"micro_f1": 0.9545454545454546, "macro_f1": 0.9520697167755992}, {"micro_f1": 0.9540229885057472, "macro_f1": 0.9538706256627785}, {"micro_f1": 0.9885057471264368, "macro_f1": 0.9884996695307335}], "sine": [{"micro_f1": 0.5795454545454546, "macro_f1": 0.5589868617093322}, {"micro_f1": 0.45454545454545453, "macro_f1": 0.4358974358974359}, {"micro_f1": 0.3409090909090909, "macro_f1": 0.3354166666666667}, {"micro_f1": 0.4942528735632184, "macro_f1": 0.44873271889400923}, {"micro_f1": 0.40229885057471265, "macro_f1": 0.39258861439312565}]}, "107": {"stem": [{"micro_f1": 0.9662921348314607, "macro_f1": 0.9661382371591629}, {"micro_f1": 0.9887640449438202, "macro_f1": 0.9878829135466304}, {"micro_f1": 0.9886363636363636, "macro_f1": 0.9883828382838282}, {"micro_f1": 0.9431818181818182, "macro_f1": 0.9431744801756425}, {"micro_f1": 1.0, "macro_f1": 1.0}], "sine": [{"micro_f1": 0.6853932584269663, "macro_f1": 0.6734800838574424}, {"micro_f1": 0.7415730337078652, "macro_f1": 0.72782874617737}, {"micro_f1": 0.625, "macro_f1": 0.6166336633663365}, {"micro_f1": 0.7272727272727273, "macro_f1": 0.7201907790143085}, {"micro_f1": 0.875, "macro_f1": 0.8745951548127996}]}}

In [94]:
d = {"104": {"stem": [{"micro_f1": 1.0, "macro_f1": 1.0}, {"micro_f1": 0.9662921348314607, "macro_f1": 0.9631469979296066}, {"micro_f1": 0.9775280898876404, "macro_f1": 0.9775252525252525}, {"micro_f1": 1.0, "macro_f1": 1.0}, {"micro_f1": 0.9659090909090909, "macro_f1": 0.9658694246929541}], "sine": [{"micro_f1": 0.5280898876404494, "macro_f1": 0.34558823529411764}, {"micro_f1": 0.8314606741573034, "macro_f1": 0.8026607538802661}, {"micro_f1": 0.48314606741573035, "macro_f1": 0.32575757575757575}, {"micro_f1": 0.5393258426966292, "macro_f1": 0.4073412376157219}, {"micro_f1": 0.44318181818181823, "macro_f1": 0.35206611570247937}]}, "106": {"stem": [{"micro_f1": 0.9431818181818182, "macro_f1": 0.9431744801756425}, {"micro_f1": 0.9659090909090909, "macro_f1": 0.9629161399072903}, {"micro_f1": 0.9431818181818182, "macro_f1": 0.9397507873476654}, {"micro_f1": 0.9310344827586207, "macro_f1": 0.9305851063829786}, {"micro_f1": 0.9885057471264368, "macro_f1": 0.9884996695307335}], "sine": [{"micro_f1": 0.5113636363636364, "macro_f1": 0.4874712176621969}, {"micro_f1": 0.6136363636363636, "macro_f1": 0.61343669250646}, {"micro_f1": 0.3181818181818182, "macro_f1": 0.30047694753577103}, {"micro_f1": 0.45977011494252873, "macro_f1": 0.4067894965907442}, {"micro_f1": 0.40229885057471265, "macro_f1": 0.3653198653198653}]}}

In [91]:
d = {"106": {"stem": [{"micro_f1": 0.9659090909090909, "macro_f1": 0.9659046881053854}, {"micro_f1": 0.9772727272727273, "macro_f1": 0.9750990379173741}, {"micro_f1": 0.9886363636363636, "macro_f1": 0.9881958417169685}, {"micro_f1": 0.9655172413793104, "macro_f1": 0.9654441943598571}, {"micro_f1": 0.9770114942528736, "macro_f1": 0.9770084566596196}], "sine": [{"micro_f1": 0.6022727272727273, "macro_f1": 0.6018099547511313}, {"micro_f1": 0.625, "macro_f1": 0.6245636716224952}, {"micro_f1": 0.39772727272727276, "macro_f1": 0.38429042904290434}, {"micro_f1": 0.5402298850574713, "macro_f1": 0.5172031076581576}, {"micro_f1": 0.47126436781609193, "macro_f1": 0.4550653594771242}]}}

In [95]:
metric = 'micro_f1'
for dataset in d.keys():
    for method_name in d[dataset]:
        #print(d[dataset])
        arr = d[dataset][method_name]
        arr = np.array(list(map(lambda x: x[metric], arr)))
        print(f'{dataset}-{method_name} Mean: {np.mean(arr)}, Std: {np.std(arr)}')
            
        

104-stem Mean: 0.9819458631256385, Std: 0.015320820536590759
104-sine Mean: 0.5650408580183861, Std: 0.13752275228056404
106-stem Mean: 0.954362591431557, Std: 0.020461166956798775
106-sine Mean: 0.46105015673981187, Std: 0.09967819562207558


In [101]:
0.982 - sqrt(0.015)

0.85952551286084111

In [102]:
0.982 + sqrt(0.015)

1.104474487139159

In [103]:
0.015 ** 2

0.000225